# 정규 표현식을 이용한 corpus 생성
---

In [43]:
import os, re
import numpy as np
import tensorflow as tf
import glob

txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

#여러 개의 txt파일을 모두 읽어서 raw_corpus에 넣자
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I hear you callin\', "Here I come baby"', 'To save you, oh oh', "Baby no more stallin'"]


# 데이터 정제
---

In [44]:
# 입력된 문장을
#       1. 소문자로 바꾸고, 양쪽 공백을 지운다.
#       2. 특수문자 양쪽에 공백을 넣고
#       3. 여러개의 공백은 하나의 공백으로 바꾼다
#       4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#       5. 다시 양쪽 공백을 지운다
#       6. 문장 시작에는 <start>, 끝에는 <end>를 추가한다.
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있다!

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    if sentence: # 비어있는 sentence 제외
        sentence = '<start> ' + sentence + ' <end>' # 6
        return sentence

corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 corpus에 포함하지 않는다
    if len(sentence) == 0: continue 
    if sentence[-1] == ":": continue
    # preprocess_sentence() 리턴값이 비어있거나 나온 결과 값을 split하여 길이가 15이상이면 건너뛴다
    if preprocess_sentence(sentence) == None or len(preprocess_sentence(sentence).split(' ')) > 15:
        continue
    preprocessed_sentence = preprocess_sentence(sentence) 
    corpus.append(preprocessed_sentence)

corpus[:10]

['<start> i hear you callin , here i come baby <end>',
 '<start> to save you , oh oh <end>',
 '<start> baby no more stallin <end>',
 '<start> these hands have been longing to touch you baby <end>',
 '<start> and now that you ve come around , to seein it my way <end>',
 '<start> it s unbelieveable how your body s calling for me <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you tell me , what s your desire <end>',
 '<start> baby your wish is my deal oh yes it is baby <end>']

# tf.keras.preprocessing.text.Tokenizer를 이용한 corpus를 텐서로 변환
---

In [45]:
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만든다
    # 우리는 이미 문장을 정제했으니 filer가 필요 없다
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾼다
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 단어장의 크기
        filters=' ',
        oov_token = "<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성한다.
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환한다
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰준다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용한다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15) 
    return tensor, tokenizer
 
tensor, tokenizer = tokenize(corpus)

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx > 10: break;

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it


# 평가 데이터셋 분리
---

In [46]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(tensor[:, :-1], tensor[:, 1:], test_size=0.2, shuffle=True, random_state=7)

print("enc_train shape:", enc_train.shape)
print("enc_val shape:", enc_val.shape)
print("dec_train shape:", dec_train.shape)
print("dec_val shape:", dec_val.shape)

enc_train shape: (124768, 14)
enc_val shape: (31192, 14)
dec_train shape: (124768, 14)
dec_val shape: (31192, 14)


# train, val Dataset building
---

In [47]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer 가 구축한 단어사전 내 14000개, 여기 포함되지 않은 0:<pad>를 포함하여 120001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset

#train data 데이터셋화
dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
print('train : ', dataset_train)

BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 256
steps_per_epoch = len(enc_val) // BATCH_SIZE

#val data 데이터셋화
dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)
print('val : ', dataset_val)

train :  <BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>
val :  <BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


# 모델 만들기
---

In [48]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # EMbedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) # 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) 
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
#embedding_size = 256
#hidden_size = 1024
embedding_size = 170
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)
print(model)

# 모델 학습
---

In [49]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset_train, validation_data=dataset_val, epochs=10)


Epoch 1/10
487/487 [==============================] - 487s 961ms/step - loss: 3.9206 - val_loss: 2.9859
Epoch 2/10
487/487 [==============================] - 471s 968ms/step - loss: 2.8943 - val_loss: 2.7245
Epoch 3/10
487/487 [==============================] - 472s 969ms/step - loss: 2.5728 - val_loss: 2.5472
Epoch 4/10
487/487 [==============================] - 471s 967ms/step - loss: 2.2710 - val_loss: 2.4147
Epoch 5/10
487/487 [==============================] - 471s 968ms/step - loss: 1.9875 - val_loss: 2.3156
Epoch 6/10
487/487 [==============================] - 471s 967ms/step - loss: 1.7340 - val_loss: 2.2436
Epoch 7/10
487/487 [==============================] - 471s 967ms/step - loss: 1.5150 - val_loss: 2.1963
Epoch 8/10
487/487 [==============================] - 471s 966ms/step - loss: 1.3324 - val_loss: 2.1729
Epoch 9/10
487/487 [==============================] - 471s 967ms/step - loss: 1.1933 - val_loss: 2.1729
Epoch 10/10
487/487 [==============================] - 471s 967m

In [29]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 단어 하나씩 예측해 문장을 만듭니다.
    #     1. 입력받은 문장의 텐서를 입력합니다
    #     2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #     3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #     4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다.
    while True:
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
            
    generated =""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환한다
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

## TEST
---

In [58]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so , <end> '

In [51]:
generate_text(model, tokenizer, init_sentence="<start> rain", max_len=20)

'<start> rain on me a million <end> '

In [57]:
generate_text(model, tokenizer, init_sentence="<start> he sleep", max_len=20)

'<start> he sleep , he umm and he learns he he <end> '

In [60]:
generate_text(model, tokenizer, init_sentence="<start> she go")

'<start> she go make the crew you want to find me <end> '

In [53]:
generate_text(model, tokenizer, init_sentence="<start> i hate")

'<start> i hate you but i love you <end> '

In [54]:
generate_text(model, tokenizer, init_sentence="<start> she like")

'<start> she like the way the dough fold up rolls roll up <end> '

In [61]:
generate_text(model, tokenizer, init_sentence="<start> he hate")

'<start> he hate the way i walked out the door <end> '

In [62]:
generate_text(model, tokenizer, init_sentence="<start> he late")

'<start> he late as he <unk> <end> '

In [63]:
generate_text(model, tokenizer, init_sentence="<start> back home")

'<start> back home in the back while a nigga <unk> <end> '

## 회고
___

- 이번 프로젝트에서 **어려웠던 점,**
    * 학습 시간과의 싸움
    * 문장의 완성도를 위해 데이터 전처리 과정(정규식, 토큰 15개 이상 삭제, 비어있는 문장 삭제 등등..)
    * 단어장의 크기(num_words) 12000, 학습 횟수(epoch) 10회 이하, 학습 데이터 개수 124960 이하 조건으로 Validation loss 2.2 수치를 만들기가 매우 힘들었다.
- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
    * NLP에서 전처리의 중요성, 학습 횟수, 하이퍼 파라미터의 중요성을 알았다. 
    * 하이퍼 파라미터를 너무 높게 설정 시, 시간도 오래 걸리게 되고 오버 피팅이 발생할 수 있다.
    * 하이퍼 파라미터 (embedding_size, hidden_size) 모델 학습 과정에서 어떤식으로 구동되는지 모호하다.
- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.
    * 데이터 전처리 과정에서 문장의 완결성 및 학습 데이터 갯수의 조건을 맞추기 위해 정규화 이후 문장을 분석하면서 데이터 정규식 표현에서 여러가지를 시도했었다.
    * 하이퍼 파라미터 (embedding_size, hidden_size) 를 조절해나가며 validation loss 2.2 이하로 맞추기 위해 노력하였다.
- **자기 다짐**
    * NLP에서 데이터 전처리 과정이 굉장히 중요하니 데이터 분석에 시간을 많이 투자하여 완벽한 데이터를 만든 후 모델링을 시작해야 된다.
    * 다음부터는 내가 진행한 테스트에 대해 기록을 하면서 진행하도록 하자. (그래프로 표현하도록 해보자)